In [15]:
from langgraph.graph import StateGraph, START, END
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [3]:
# We interact with LLM in the form of Messages like Human Messages, AI Messages, Tool Messages.
# We will use reducer so that previous value should not get replaced, or deleted.
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
llm= ChatOpenAI()

In [5]:
def chat_node(state: ChatState):
    messages=state['messages']

    res= llm.invoke(messages)

    return {'messages': [res]}


In [16]:
checkpointer= MemorySaver()

graph= StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

wf= graph.compile(checkpointer=checkpointer)

In [18]:
thread_id='1'

while True:
    user_message= input("Type your Message Here:")
    print("User: ",user_message)

    if user_message.strip().lower() in ['exit', 'thank you', 'bye', 'quit']:
        break

    config= {'configurable':{'thread_id': thread_id}}

    res= wf.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)

    print('LIRA: ',res['messages'][-1].content)

User:  hi Lira
LIRA:  Hello! How can I assist you today?
User:  i am a python developer and i wnatt you to help me in simple task
LIRA:  Of course! I'd be happy to help you with a simple Python task. Please let me know what you need assistance with.
User:  ok
LIRA:  Feel free to provide more details whenever you're ready, and I'll do my best to assist you with your Python task.
User:  2+3=5 prove it
LIRA:  In Python, you can use the following code to prove that 2+3 equals 5:

```python
result = 2 + 3
print(result)
```

When you run this code, the output will be `5`, confirming that 2+3 does indeed equal 5. Let me know if you need any further assistance!
User:  who am i?
LIRA:  You are Navin, a Python developer seeking assistance with a task. How can I help you further with your programming work?
User:  who are you?
LIRA:  I am an AI assistant here to help and provide support with any questions or tasks you may have. How can I assist you today?
User:  what is your name?
LIRA:  I am a la

In [19]:
wf.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='23142ed5-e036-4c76-b5a5-63e1d860893f'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0ooRluIpXcnfKl9Udilw4Hd67vCb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--625d30e9-276a-4a26-ab26-577305785329-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='i am navin i